In [ ]:
# logits = tf.matmul(X,W) + b
# hypothesis = tf.nn.softmax(logits)

# tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = labels)  : logits 값을 input으로 받음


In [2]:
import tensorflow as tf
import numpy as np

In [93]:
xy = np.loadtxt('/content/drive/MyDrive/data-04-zoo.csv', delimiter = ',', dtype = np.float32)

In [94]:
print(xy)
xy.shape

[[1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 3.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 6.]
 [0. 1. 1. ... 0. 0. 1.]]


(101, 17)

In [95]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [96]:
nb_classes = 7 # 0 ~ 6

# one hot encoding
# tf.one_hot(라벨 인덱스 목록, 라벨 개수)

Y_one_hot = tf.one_hot(y_data, nb_classes)# one hot rank : 1차원 추가됨 shape = (?,1,7)

In [97]:
y_data.shape

(101, 1)

In [98]:
Y_one_hot.shape

TensorShape([101, 1, 7])

In [99]:
# shape에 -1를 입력하면 고정된 차원은 우선 채우고(nb_classes) 남은 부분을 알아서 채워준다.
# [-1, nb_classes] : rank = 2 

Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

In [100]:
Y_one_hot.shape

TensorShape([101, 7])

In [107]:
# Weights and bias setting
W = tf.Variable(tf.random.normal([16,nb_classes]), name = 'weight')
b = tf.Variable(tf.random.normal([nb_classes]), name = 'bias')

def logit_fn(features) :
  return tf.matmul(features,W) + b

# accuracy 확인을 위해 hypothesis 정의
def hypothesis(features) :
  return tf.nn.softmax(logit_fn(features))

def cost_fn(features,labels) :
  logits = logit_fn(features)
  cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels)
  cost = tf.reduce_mean(cost_i)

  return cost

def grad_fn(features, labels) :
  with tf.GradientTape() as tape :
    loss = cost_fn(features, labels)
    grads = tape.gradient(loss, [W,b])
  return grads

def prediction(features, labels) :
  pred = tf.argmax(hypothesis(features),1)
  correct_prediction = tf.equal(pred, tf.argmax(labels,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy




In [108]:
def fit(features, labels, epochs=1000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1)

    for i in range(epochs):
        grads = grad_fn(features, labels)
        optimizer.apply_gradients(zip(grads, [W,b]))
        if (i==0) | ((i+1)%verbose==0):
#             print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            acc = prediction(features, labels).numpy()
            loss = cost_fn(features, labels).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))


In [109]:
fit(x_data, Y_one_hot)

Steps: 1 Loss: 5.728363990783691, Acc: 0.20792078971862793
Steps: 100 Loss: 0.6460839509963989, Acc: 0.7821782231330872
Steps: 200 Loss: 0.375124454498291, Acc: 0.9009901285171509
Steps: 300 Loss: 0.2742172181606293, Acc: 0.9207921028137207
Steps: 400 Loss: 0.21724514663219452, Acc: 0.9603960514068604
Steps: 500 Loss: 0.18015071749687195, Acc: 0.9702970385551453
Steps: 600 Loss: 0.15384608507156372, Acc: 0.9702970385551453
Steps: 700 Loss: 0.13410501182079315, Acc: 0.9801980257034302
Steps: 800 Loss: 0.11871985346078873, Acc: 0.9801980257034302
Steps: 900 Loss: 0.10641277581453323, Acc: 0.9900990128517151
Steps: 1000 Loss: 0.0963730588555336, Acc: 0.9900990128517151
